In [6]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
import yfinance as yf
from datetime import datetime
from typing import TypedDict
import time
import re

# workflow simple design
[Trigger]</br>
     ↓</br>
[Discovery Layer] </br>
     ↓ </br>
[Pre-Filter Layer] </br>
     ↓ </br>
[Deep Analysis Layer] </br>
     ↓ </br>
[Risk Management Layer]  </br>
     ↓ </br>
[Portfolio Agent] </br>
     ↓ </br>
BUY? (with confidence score) </br>
     ↓ </br>
[Alert Agent] </br>
     ↓ </br>
⏸ WAIT FOR HUMAN (with timeout) </br>
     ↓ </br>
YES ──→ [Execution Engine] → [Broker] </br>
NO  ──→ [Log & End] → [Feedback to Discovery] </br>
     ↓ </br>
[Memory Agent] ← Store execution data </br>
     ↓ </br>
[Performance Analytics] ← New component </br>
     ↓ </br>
[Confirmation Agent] </br>
     ↓ </br>
[Final Report Agent] </br>

In [ ]:
model = ChatGroq(model = 'llama-3.1-8b-instant',temperature=0.3, max_tokens = 500)
search_tool = TavilySearch(max_results = 5)

Layer 1 -- [Trigger]

In [30]:
class StockState(TypedDict):
    trigger_type: str
    trigger_source: str
    trigger_time: str
    query: str

    search_result: list
    tickers: list

Trigger for firing agentic search and for buy and sale

schedule trigger

In [ ]:
def scheduled_triigger(state:StockState):
    now = datetime.now()
    hour = now.hour
    if 9 <=hour <11:
        session="market open"
        query = "Top gaining stocks at market open today US"
    elif 11 <=hour<14:
        session = "mid day"
        query = "Top momentum stocks midday trading US market"
    elif 14<=hour <16:
        session = 'pre close'
        query = "Top stocls gaining volume per-market close US"
    else:
        session = "after-hurs"
        query   = "After hours top gaining stocks US market today"
    
    print(f" Scheduled Trigger fired - session: {session}")
    return{
        "trigger_type": "scheduled",
        "trigger_source": f"corn job - {session} session",
        "trigger_time": now.strftime("%Y-%m-%d %H:%M:%S"),
        "query": query
    }

manual trigger

In [12]:
def manual_trigger(query:str) -> dict:
    "for manual query and use  when user want specific search"
    print(f" manual trigger fired - query: '{query}'")
    return{
        "trigger_type": "manual",
        "trigger_source": "user input",
        "trigger_time" : datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "query": query
    }

price alert trigger

In [18]:
import yfinance as yf
import pandas as pd
all_history = []
WATCHLIST = ["AAPL", "TSLA", "NVDA", "MSFT", "AMD", "META", "AMZN"]
all_history = []

for ticker in WATCHLIST:
   
    df = yf.Ticker(ticker).history(period='2d')
    df['Ticker']=ticker
    all_history.append(df)


In [21]:
hist  = pd.concat(all_history)

In [25]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
Date,,,,,,,,
2026-02-26 00:00:00-05:00,274.950012,276.109985,270.799988,272.950012,32345100,0.0,0.0,AAPL
2026-02-27 00:00:00-05:00,272.809998,272.809998,262.890015,264.179993,72239400,0.0,0.0,AAPL
2026-02-26 00:00:00-05:00,414.420013,416.809998,403.660004,408.579987,53602500,0.0,0.0,TSLA
2026-02-27 00:00:00-05:00,402.940002,407.119995,398.109985,402.510010,56760900,0.0,0.0,TSLA
2026-02-26 00:00:00-05:00,194.270004,194.289993,184.320007,184.889999,360807900,0.0,0.0,NVDA
2026-02-27 00:00:00-05:00,181.250000,182.589996,176.380005,177.190002,310523200,0.0,0.0,NVDA
2026-02-26 00:00:00-05:00,404.709991,407.489990,398.739990,401.720001,34405900,0.0,0.0,MSFT
2026-02-27 00:00:00-05:00,390.880005,396.820007,389.880005,392.739990,51276300,0.0,0.0,MSFT
2026-02-26 00:00:00-05:00,208.800003,209.789993,201.460007,203.679993,35020500,0.0,0.0,AMD


In [23]:
closes = hist["Close"].dropna()

In [24]:
closes.iloc[-2]

np.float64(207.9199981689453)

In [27]:
def price_alert_trigger(state: StockState):
    """
    scan a watchlist every N minutes and fire only when a stock moves more than a threshold %

    """
    WATCHLIST = ["AAPL", "TSLA", "NVDA", "MSFT", "AMD", "META", "AMZN"]
    # treshold point for trigger if any stock moves ±2% or more
    THRESHOLD_PCT = 2.0
    alerted_tickers = []

    for ticker in WATCHLIST:
        try:
            hist = yf.Ticker(ticker).history(period = "2d")
            if len(hist)<2:
                continue
            closes = hist['Close'].dropna()
            prev = closes.iloc[-2]
            curr = closes.iloc[-1]
            pct = ((cur - prev)/prev)*100
            
            if abs(pct) >=THRESHOLD_PCT:
                direction  = "📈 UP" if pct > 0 else "📉 DOWN"
                print(f"🚨 Price Alert: {ticker} moved {pct:.2f}% {direction}")
                alerted_tickers.append(ticker)
        except Exception as e:
            print(f"❌ Price check failed for {ticker}: {e}")
            continue
    if alerted_tickers:
        query = f"Latest news and analysis failed for {' '.join(alerted_tickers)}"
    else:
        query = "Top stocks increasing today in US market"
    return {
        "trigger_type":'price_alert',
        "trigger_source": f"watchlist monitor — alerted: {alerted_tickers or 'none'}",
        "trigger_time":   datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "query":          query
    }
            



In [ ]:
def news_alert_trigger(state: StockState):
    """
    searches for breaking market news and fires when high-impact keywords are detected in headlines.
    """
    HIGH_IMPACT_KEYWORDS = [
        "earnings beat", "FDA approval", "merger", "acquisition",
        "short squeeze", "all-time high", "bankruptcy", "IPO",
        "fed rate", "market crash", "record high"
    ]

    try:
        results  = search_tool.invoke({"query": "breaking stock market news today"})
        combined = " ".join([r.get("content", "") for r in results]).lower()

        matched  = [kw for kw in HIGH_IMPACT_KEYWORDS if kw in combined]
    except:
        matched  = []

    if matched:
        print(f"📰 News Alert triggered — keywords: {matched}")
        query = f"Stocks affected by {', '.join(matched[:3])} today"
    else:
        print("📰 News Alert: No high-impact news found. Using default query.")
        query = "Top gaining stocks today US market"

    return {
        "trigger_type":   "news_alert",
        "trigger_source": f"news monitor — keywords: {matched or 'none'}",
        "trigger_time":   datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "query":          query
    }

In [29]:
def trigger_node(state: StockState):
    """
    it is master trigger node to change trigger_mode to switch behavious
     options scheduled,manual,price_alert and news_alert

    """
    TRIGGER_MODE = "scheduled"   

    if   TRIGGER_MODE == "scheduled":   return scheduled_trigger(state)
    elif TRIGGER_MODE == "price_alert": return price_alert_trigger(state)
    elif TRIGGER_MODE == "news_alert":  return news_alert_trigger(state)
    else:                               return scheduled_trigger(state)

Layer -2 DISCOVERY

In [31]:
def search_storcks(state:StockState):
    #runs the web search using the query built by trigger
    printf('\n Searching: "{state["query"]}"')
    results = search_tool.invoke({"query":state['query']})
    print(f" -> {len(results)} results found")
    return {"search_result": results}

def extract_tickers(state: StockState):
    """Extracts all ticker symbols from search results using regex."""
    content     = str(state["search_result"])
    raw_tickers = re.findall(r'\b([A-Z]{2,5})\b', content)

    blacklist = {
        "USD","TTM","YOY","EPS","DIV","ETF","NYSE","NASDAQ","SEC","CEO",
        "CFO","IPO","API","URL","AI","US","PE","AM","PM","EST","THE",
        "FOR","AND","INC","LLC","LTD","AVG","VOL","BUY","SELL","STRONG",
        "NO","MKT","NA","SIGN","TOP","NEW","OLD","HIGH","LOW","NET",
        "COM","ORG","HTML","JSON","HTTP","HTTPS","GET","POST","MO","YR",
        "YTD","QOQ","EMA","SMA","RSI","ATR","GDP","CPI","FED","IMF"
    }

    tickers = list(dict.fromkeys([t for t in raw_tickers if t not in blacklist]))
    print(f"   → Extracted {len(tickers)} tickers: {tickers}")
    return {"tickers": tickers}